# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

In [151]:
#! wget https://www.dropbox.com/s/r0q0p0uprhcp8bb/train-sample.zip
#! unzip train-sample.zip

In [152]:
#! wget https://www.dropbox.com/s/50vw2gsglc91f6o/train.zip
#! unzip train.zip

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [85]:
import csv

INPUT_DATA = 'train.csv'
#INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

{'BodyMarkdown': "I'm new to C#, and I want to use a trackbar for the forms opacity\nThis is my code\n\n    decimal trans = trackBar1.Value / 5000\n    this.Opacity = trans\n\nWhen I try to build it, I get this error\n\n**Cannot implicitly convert type 'decimal' to 'double**\n\nI tried making trans a double, but then the control doesn't work. This code worked fine for me in VB.NET. Any suggestions?",
 'OpenStatus': 'open',
 'OwnerCreationDate': '07/31/2008 21:33:24',
 'OwnerUndeletedAnswerCountAtPostTime': '0',
 'OwnerUserId': '8',
 'PostClosedDate': '',
 'PostCreationDate': '07/31/2008 21:42:52',
 'PostId': '4',
 'ReputationAtPostCreation': '1',
 'Tag1': 'c#',
 'Tag2': '',
 'Tag3': '',
 'Tag4': '',
 'Tag5': '',
 'Title': 'Decimal vs Double?'}

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [86]:
from tqdm import tqdm_notebook

STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, train_output='train', test_output='test', ytest_output='ytest'):
    reader = csv.DictReader(open(INPUT_DATA, encoding='utf-8'))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in tqdm_notebook(reader):
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = '%s %s\n' % (label, features)
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [87]:
import re

def extract_title(row):
    title = row['Title']
    words = re.findall('[A-Za-z\']+', title)
    for i in range(len(words)):
        words[i] = words[i].lower()
    return ' '.join(words)

data2vw(lambda row: '| %s' % extract_title(row))
! head -n 5 train

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


4 | decimal vs double
4 | percentage width child in absolutely positioned parent doesn't work in ie
4 | tools for porting j code to c
4 | throw error in mysql trigger
4 | what's the difference between math floor and math truncate


Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвать `vw --help`.

In [145]:
! vw -d train --loss_function logistic --oaa 5 -f model

final_regressor = model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4        1        4
0.500000 0.000000            2            2.0        4        4       12
0.250000 0.000000            4            4.0        4        4        6
0.125000 0.000000            8            8.0        4        4        7
0.062500 0.000000           16           16.0        4        4        5
0.062500 0.062500           32           32.0        4        4        9
0.046875 0.031250           64           64.0        4        4        5
0.062500 0.078125          128          128.0        4        4       11
0.089844 0.117188          256          256.0        4        4        8
0.070312 0.050781          512          512.0   

Применим модель к тестовой выборке и сохраним предсказания.

In [146]:
! vw -i model -t test -r pred

only testing
raw predictions = pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        4        4        9
0.000000 0.000000            2            2.0        4        4        7
0.000000 0.000000            4            4.0        4        4        8
0.125000 0.250000            8            8.0        4        4        6
0.125000 0.125000           16           16.0        4        4        7
0.156250 0.187500           32           32.0        4        4       10
0.093750 0.031250           64           64.0        4        4        8
0.117188 0.140625          128          128.0        4        4        8
0.121094 0.125000          256          256.0        4        4       15
0.103516 0.085938          512       

In [126]:
! head -n 5 pred

1:-3.96212 2:-5.91553 3:-4.17684 4:2.434 5:-6.30902
1:-7.23647 2:-7.77868 3:-6.88226 4:4.5498 5:-6.60568
1:-3.83466 2:-5.41 3:-5.04316 4:2.91026 5:-6.61622
1:-3.88813 2:-6.19204 3:-6.32849 4:2.96315 5:-6.38496
1:-5.65263 2:-3.88128 3:-3.87864 4:2.01935 5:-6.2923


Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [128]:
import numpy as np

def count_probas(pred_values):
    exps = np.exp(pred_values)
    exp_sum = np.sum(exps)
    return exps / exp_sum

def get_scores(ytest_input='ytest', pred_input='pred'):
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')
    
    for label, pred in zip(reader_ytest, reader_pred):
        temp = pred.split(' ')
        pred_values = np.array([float(item.split(':')[1]) for item in temp])
        n += 1
        if int(label) - 1 != np.argmax(pred_values):
            error += 1
        probas = count_probas(pred_values)
        loss += np.log(probas[int(label) - 1])
        # YOUR CODE HERE
        
    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [129]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14658
accuracy = 0.97791


На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [149]:
import random

best_loss, accur = get_scores()
best = (0, 0, 0)
ngram_param = [0, 1, 2, 3, 4]
skips_param = [0, 1, 2, 3, 4]
affix_param = [-1, +1, 0, -2, +2]
for i in range(5):
    ngr = random.choice(ngram_param)
    skp = random.choice(skips_param)
    aff = random.choice(affix_param)
    ! vw -d train --loss_function logistic --oaa 5 --ngram ngr --skips skp --affix aff -f model --quiet
    ! vw -i model -t test -r pred --quiet
    loss, accur = get_scores()
    if best_loss > loss:
        best = (ngr, skp, aff)
        best_loss = loss

vw (parse_args.cc:250): malformed affix argument (length must be 1..7): aff
vw (parse_args.cc:250): malformed affix argument (length must be 1..7): aff
vw (parse_args.cc:250): malformed affix argument (length must be 1..7): aff
vw (parse_args.cc:250): malformed affix argument (length must be 1..7): aff
vw (parse_args.cc:250): malformed affix argument (length must be 1..7): aff


In [150]:
print('Best loss:')
print(best_loss)
print('Best parameters:')
print(best)

Best loss:
0.146578691026
Best parameters:
(0, 0, 0)


Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [ ]:
# YOUR CODE HERE

Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [104]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Version 8.1.1
Min label:-50.000000
Max label:50.000000
bits:18
lda:0
0 ngram: 
0 skip: 
options: --oaa 5
Checksum: -830150891
:0
':82960:-1.101947
'':145776:-1.849990
''':183184:-0.954318
'''':221848:-0.855841
''''[1]:221849:-0.787449
''''[2]:221850:-0.899348
''''[3]:221851:0.162724
''''[4]:221852:-0.190776
'''[1]:183185:-0.814208
'''[2]:183186:-1.086044
'''[3]:183187:1.082939
'''[4]:183188:-0.939128
''[1]:145777:-1.254720
''[2]:145778:-1.245932
''[3]:145779:1.351760
''[4]:145780:-1.752311
''assembler'':183600:-0.296368
''assembler''[1]:183601:-0.374649
''assembler''[2]:183602:-0.444131
''assembler''[3]:183603:0.483609


Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

In [140]:
def max_weight_features(n_features, class_num, model_input = 'model.readable'):
    h = []
    line_count = 0
    model = open(model_input, 'r')
    for feature in model:
        if line_count > 9:
            params = feature.strip().split(':')
            if class_num != 0:
                if len(params[0]) > 1 and params[0][-2] == str(class_num):
                    heapq.heappush(h, (abs(float(params[2])), float(params[2]), 
                                       params[0], int(params[1])))
                    if len(h) > n_features:
                        heapq.heappop(h)
            else:
                if params[0][-1] != ']':
                    heapq.heappush(h, (abs(float(params[2])), float(params[2]), 
                                       params[0], int(params[1])))
                    if len(h) > n_features:
                        heapq.heappop(h)
        else:
            line_count += 1
    return list(reversed(sorted(h)))

In [142]:
for i in range(5):
    maxw_features = max_weight_features(10, i)
    print('Max weighted features for {}-th class:'.format(i))
    for feature in maxw_features:
        print(feature)

Max weighted features for 0-th class:
(4.665564, -4.665564, 'uitableviewcell', 261952)
(4.665564, -4.665564, 'templatingengine', 261952)
(4.665564, -4.665564, 'simplink', 261952)
(4.665564, -4.665564, 'runtimeinvisibleparameterannotations', 261952)
(4.665564, -4.665564, 'gtkviewtree', 261952)
(4.665564, -4.665564, 'businessconnectornet', 261952)
(4.574079, -4.574079, 'robotlegs', 15552)
(4.574079, -4.574079, "eq'", 15552)
(4.574079, -4.574079, 'coregrahpics', 15552)
(4.574079, -4.574079, 'compund', 15552)
Max weighted features for 1-th class:
(3.047302, -3.047302, 'uploading[1]', 72969)
(3.047302, -3.047302, 'namelength[1]', 72969)
(3.047302, -3.047302, 'mysqldb[1]', 72969)
(3.047302, -3.047302, 'getpropertyaction[1]', 72969)
(3.047302, -3.047302, 'eacel[1]', 72969)
(3.047302, -3.047302, 'clearcacheinterceptor[1]', 72969)
(3.006469, -3.006469, 'overlay[1]', 67377)
(2.968956, -2.968956, 'strange[1]', 260657)
(2.968956, -2.968956, 'reorganization[1]', 260657)
(2.968956, -2.968956, 'pth[1

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Придумайте и извлеките признаки из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [119]:
from collections import Counter, defaultdict

def extract_ngram_body(row, ngram=3):
    mark_down = row['BodyMarkdown']
    words = re.findall('[A-Za-z\']+', title)
    for i in range(len(words)):
        words[i] = words[i].lower()
    ngrams = Counter()
    for word in words:
        for i in range(len(word) - ngram + 1):
            ngrams[word[i:i+ngram]] += 1
    return ngrams

def extract_tags(row):
    tags = defaultdict(str)
    for i in range(1, 6):
        tags['Tag' + str(i)] = row['Tag' + str(i)]
    return tags
    # YOUR CODE HERE

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [ ]:
extractors_list = [
    ('t', extract_title), 
    ('b', extract_ngram_body), 
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
    return feature_extractor
    
    
data2vw(make_feature_extractor(extractors_list))

! head -n 1 train

Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. <br>
Выберите не менее трех параметров (и дискретные, и непрерывные). Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений в цикле, но лучше воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Удобночитаемый вывод и/или графики для разных значений параметров обязательны.

Совсем необязательно добиваться наилучшего качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.


Помогло ли добавление квадратичных взаимодействий? Каких групп признаков? Какие параметры повлияли на улучшение качества сильнее всего?

In [ ]:
# YOUR CODE HERE

Приблизительная разбалловка:
* 3 балла в верно реализованные функции
* 1 балл за ответ на вопрос про параметры `--passes` и `-b` (с графиком/выводом)
* 2 балла за эффективное нахождение наиболее значимых признаков
* 1 балл за добавление своих признаков
* 3 балла за финальную часть (квадратичные взаимодействия, подбор параметров, графики/вывод и комментарии)